# Collect array

Here, we take a closer look at functions that vary across SQL dialects, allowing columns in a database to be transformed into an array that fits in one table cell.

The following cell starts the databases we'll use in Docker. For now, they are:

- PostgreSQL
- ClickHouse

In [14]:
docker run --rm -d\
    -e POSTGRES_PASSWORD=docker_app \
    --name collect_array_postgres_examples \
    postgres:15.4 &> /dev/null
docker run --rm -d \
    --name collect_array_click_examples \
    clickhouse/clickhouse-server &> /dev/null
sleep 5

**Note**: Don't forget to stop all containers once finished.

In [13]:
docker stop collect_array_postgres_examples collect_array_click_examples

collect_array_postgres_examples
collect_array_click_examples


## Clickhouse syntax

In ClickHouse, for such transformations, you need to use the `groupArray` function.

---

The following cell shows the application of the `groupArray` function to the `some_value` column, along with extracting specific positions from the resulting array.

In [9]:
docker exec collect_array_click_examples clickhouse-client -q "
CREATE TEMPORARY TABLE temp (some_value Int32) ENGINE = Memory;

INSERT INTO temp VALUES
    (4), (3), (3), (1), (3), (3), (3), (5), (5), (1);

SELECT 
    groupArray(some_value), 
    groupArray(some_value)[1], 
    groupArray(some_value)[2] 
FROM temp
FORMAT Pretty;
"

   ┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
   ┃ groupArray(a)         ┃ arrayElement(groupArray(a), 1) ┃ arrayElement(groupArray(a), 2) ┃
   ┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
1. │ [4,3,3,1,3,3,3,5,5,1] │                              4 │                              3 │
   └───────────────────────┴────────────────────────────────┴────────────────────────────────┘


## Index out of range

When trying to access an out-of-range position in this command, the common behavior across SQL dialects is to return a reserved value. However, the specific behavior can vary between different SQL dialects.

---

The following examples demonstrate attempts to access out-of-range positions in different SQL dialects.

Here is an example from PostgreSQL, which shows that PostgreSQL returns NULL in such cases.

In [24]:
docker exec -i collect_array_postgres_examples psql -U postgres -d postgres << EOF
SELECT 
    (array_agg(some_value))[5] IS NULL fifth_element_is_null
FROM (
    VALUES
    (4), (3), (3)
) AS temp(some_value);
EOF

 fifth_element_is_null 
-----------------------
 t
(1 row)



In ClickHouse, it returns reserved values to replace non-existent values. The following cell shows that instead of a non-existent element of the array, ClickHouse returns:

- `0` for numeric columns.
- An empty string `''` for string columns.

In [25]:
docker exec collect_array_click_examples clickhouse-client -q "
CREATE TEMPORARY TABLE temp (numeric_value Int, string_value String) ENGINE = Memory;

INSERT INTO temp VALUES
    (1, 'a'), (2, 'b'), (3, 'c');

SELECT 
    groupArray(numeric_value)[5] numeric_index_out,
    groupArray(string_value)[5] string_index_out
FROM temp
FORMAT Pretty;
"

   ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
   ┃ numeric_index_out ┃ string_index_out ┃
   ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
1. │                 0 │                  │
   └───────────────────┴──────────────────┘
